In [10]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

tokenizer.pre_tokenizer = Whitespace()

tokenizer.train(dataset, trainer)

TypeError: Can't convert DatasetDict({
    train: Dataset({
        features: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '1

In [1]:
import json
f = open('./train.json')
f2 = open('./test.json')
data = open('./dataset.json')
train = json.load(f)
test = json.load(f2)
dataset = data

f.close()
data.close()
f2.close()

In [2]:
train

{'0': {'row_id': 23620,
  'subject_id': 19323,
  'hadm_id': 111122,
  'readmission': 0,
  'inform': '5:38 AM. CHEST (PORTABLE AP) Clip #. Reason: r/o infiltrate. . MEDICAL CONDITION:. 86 year old man with respiratory distress. REASON FOR THIS EXAMINATION:. r/o infiltrate. . FINAL REPORT. INDICATION: 86-year-old with respiratory distress, rule out infiltrate.. . SINGLE PORTABLE UPRIGHT CHEST: Reports from two prior chest x-rays from. and are available by report only.. . Surgical clips are seen over the left upper abdomen. The patient is slightly. rotated to the right, and in addition, there is thoracic scoliosis, which. slightly limits interpretation. The heart size is within normal limits. The. mediastinal and hilar contours are unremarkable allowing for tortuous aorta.. In the medial aspect of the right lower lung zone, likely in the right middle. lobe, there is a dense area of consolidation, likely representing pneumonia.. The lungs are otherwise clear.. . IMPRESSION: Focal consolida

In [3]:
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("bvanaken/CORe-clinical-mortality-prediction")

def tokenize_function(examples):
    return tokenizer(examples["inform"], padding="1000000000000000019884624838656", truncation=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [5]:
from datasets import load_dataset

dataset = load_dataset('json' , data_files='C:/Users/170890/capstone/nlpmimic/capstone/dataset.json')

Using custom data configuration default-c1089ad83e310a8d
Reusing dataset json (C:\Users\170890\.cache\huggingface\datasets\json\default-c1089ad83e310a8d\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bvanaken/CORe-clinical-mortality-prediction")


def tokenize_function(examples):
    return tokenizer(examples["inform"], padding="1000000000000000019884624838656", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

KeyError: 'inform'

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("yelp_review_full")
dataset

In [ ]:
print(len(dataset))
print(type(dataset))